<h1>Federated Learning using MNIST</h1>

<h2>Centralized Training</h2>

In [1]:
import tensorflow as tf

# Import Datasets
mnist = tf.keras.datasets.mnist
(data_train, label_train), (data_test, label_test) = mnist.load_data()

# Data pre-processing
data_train, data_test = data_train/255.0, data_test/255.0

# Build model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])
    
# Compile model
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# Train Model
model.fit(data_train, label_train, epochs=5)

# Get results
test_loss, test_acc = model.evaluate(data_test, label_test)
print("test loss: ", test_loss)
print("test accuracy: ", test_acc)

Metal device set to: Apple M1 Max
Epoch 1/5


2022-05-21 15:15:45.056276: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-21 15:15:45.056406: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-05-21 15:15:45.208493: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  14/1875 [..............................] - ETA: 7s - loss: 1.6236 - accuracy: 0.5268  

2022-05-21 15:15:45.338093: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 7s 4ms/step - loss: 0.2008 - accuracy: 0.9405
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0797 - accuracy: 0.9754
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0528 - accuracy: 0.9841
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0372 - accuracy: 0.9878
Epoch 5/5
 43/313 [===>..........................] - ETA: 0s - loss: 0.1072 - accuracy: 0.9695

2022-05-21 15:16:21.446660: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 4ms/step - loss: 0.0784 - accuracy: 0.9758
test loss:  0.07836448401212692
test accuracy:  0.9758000373840332


<h2>Test "get_weights", "set_weights"</h2>

In [2]:
weights = model.get_weights()
print(weights)
import sys
print(sys.get)

cp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

cp_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

cp_model.set_weights(weights)

# Get results
test_loss, test_acc = model.evaluate(data_test, label_test)
# print("test loss: ", test_loss)
# print("test accuracy: ", test_acc)

[array([[-0.02964921,  0.01799724, -0.00025862, ...,  0.00495534,
         0.0315336 , -0.06313174],
       [ 0.06230609, -0.04954939,  0.05480339, ..., -0.0206655 ,
         0.04888868,  0.06787175],
       [ 0.03984915, -0.02480331,  0.0296289 , ...,  0.03744321,
        -0.0378321 ,  0.02476066],
       ...,
       [-0.03260977, -0.06802543,  0.048971  , ...,  0.01924925,
        -0.05154556, -0.01255806],
       [ 0.01120001, -0.05747771,  0.00133567, ..., -0.01201348,
         0.05898423, -0.00270942],
       [-0.04028355,  0.01779348, -0.05213377, ...,  0.05570427,
         0.0565781 ,  0.06019202]], dtype=float32), array([ 4.19299155e-02, -4.59218659e-02, -1.27966143e-02,  3.05585489e-02,
        7.14712292e-02, -3.45433541e-02,  5.17631285e-02,  1.08995013e-01,
       -3.82846072e-02, -3.06963101e-02, -1.75173506e-02, -3.38909067e-02,
       -4.89388406e-02, -3.30282678e-03, -6.36641383e-02,  6.25861734e-02,
        1.28506841e-02,  1.17554180e-02,  4.99538183e-02, -8.35589543e

313/313 [==============================] - 1s 4ms/step - loss: 0.0784 - accuracy: 0.9758


<h2>Data Split 3 of Clients</h2>

In [3]:
c1_data_train = data_train[:20000, :, :]
c1_label_train = label_train[:20000]

c2_data_train = data_train[20000:40000, :, :]
c2_label_train = label_train[20000:40000]

c3_data_train = data_train[40000:60000, :, :]
c3_label_train = label_train[40000:]

print(c1_data_train.shape, len(c1_label_train))
print(c2_data_train.shape, len(c2_label_train))
print(c3_data_train.shape, len(c3_label_train))

(20000, 28, 28) 20000
(20000, 28, 28) 20000
(20000, 28, 28) 20000


<h2>Get Weights from Client 1</h2>

In [4]:
# Train Client 1
c1_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

c1_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

c1_model.fit(c1_data_train, c1_label_train, epochs=5)

c1_weights = c1_model.get_weights()

Epoch 1/5
 22/625 [>.............................] - ETA: 2s - loss: 1.3911 - accuracy: 0.6193

2022-05-21 15:16:24.117070: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 3s 4ms/step - loss: 0.3186 - accuracy: 0.9083
Epoch 2/5
625/625 [==============================] - 2s 4ms/step - loss: 0.1301 - accuracy: 0.9614
Epoch 3/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0799 - accuracy: 0.9765
Epoch 4/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0504 - accuracy: 0.9856
Epoch 5/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0349 - accuracy: 0.9901


<h2>Get Weights from Client 2</h2>

In [5]:
# Train Client 2
c2_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

c2_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

c2_model.fit(c2_data_train, c2_label_train, epochs=5)

c2_weights = c2_model.get_weights()

Epoch 1/5
 28/625 [>.............................] - ETA: 2s - loss: 1.1973 - accuracy: 0.6819

2022-05-21 15:16:36.276181: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 3s 4ms/step - loss: 0.3285 - accuracy: 0.9066
Epoch 2/5
625/625 [==============================] - 2s 4ms/step - loss: 0.1359 - accuracy: 0.9590
Epoch 3/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0832 - accuracy: 0.9759
Epoch 4/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0546 - accuracy: 0.9844
Epoch 5/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0385 - accuracy: 0.9895


<h2>Get Weights from Client 3</h2>

In [6]:
# Train Client 3
c3_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

c3_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

c3_model.fit(c3_data_train, c3_label_train, epochs=5)

c3_weights = c3_model.get_weights()

Epoch 1/5
 40/625 [>.............................] - ETA: 2s - loss: 1.0140 - accuracy: 0.7188

2022-05-21 15:16:48.361958: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 3s 4ms/step - loss: 0.3202 - accuracy: 0.9065
Epoch 2/5
625/625 [==============================] - 2s 4ms/step - loss: 0.1349 - accuracy: 0.9593
Epoch 3/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0815 - accuracy: 0.9746
Epoch 4/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0515 - accuracy: 0.9847
Epoch 5/5
625/625 [==============================] - 2s 4ms/step - loss: 0.0356 - accuracy: 0.9890


<h2>Aggregation Weights</h2>

In [7]:
import numpy as np

agg_weights = (np.array(c1_weights) + np.array(c2_weights) + np.array(c3_weights)) / 3.0
# agg_weights = (np.array(c1_weights) + np.array(c2_weights) + np.array(c3_weights)) / 3.0

agg_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax),
])

agg_model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

agg_model.set_weights(agg_weights)

# Get results
test_loss, test_acc = agg_model.evaluate(data_test, label_test)
print("test loss: ", test_loss)
print("test accuracy: ", test_acc)

 13/313 [>.............................] - ETA: 1s - loss: 0.0678 - accuracy: 0.9808 

/var/folders/86/dxn6gj7x5jdcw5893s56tz2h0000gn/T/ipykernel_65909/4010025759.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  agg_weights = (np.array(c1_weights) + np.array(c2_weights) + np.array(c3_weights)) / 3.0
2022-05-21 15:17:00.450064: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 4ms/step - loss: 0.0855 - accuracy: 0.9737
test loss:  0.08548560738563538
test accuracy:  0.9737000465393066
